<a href="https://colab.research.google.com/github/QingZhou-YangHY/ML_HW/blob/main/ML_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN:传统的神经网络在处理数据时，是“一次性”的，每次输入都是独立的。但很多数据是序列性的，RNN 就是专门为处理这类有顺序、有上下文关联的数据而设计的神经网络。有“记忆”： RNN 的核心特点是它有一个“循环”结构。

基于语音特征进行电话（音素）分类，这本质上是一个序列分类问题。

Download Data


In [40]:
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip
!unzip -q libriphone.zip
!ls libriphone

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1
From (redirected): https://drive.google.com/uc?id=1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1&confirm=t&uuid=52a29c9e-ab5d-4cb3-ab05-0e1a81531ff2
To: /content/libriphone.zip
100% 384M/384M [00:08<00:00, 44.6MB/s]
feat  test_split.txt  train_labels.txt	train_split.txt


In [41]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

设置同一个种子方面实验复现

In [45]:
import os
import torch
from tqdm import tqdm


#从指定路径加载一个 PyTorch Tensor 特征文件。
def load_feat(path):
    feat = torch.load(path)
    return feat
#对输入的 PyTorch Tensor x 进行行（时间步）的位移操作。
#这是一个辅助函数，用于 concat_feat 中实现上下文拼接。

def shift(x, n):
    """
    shift 函数在这个代码中的作用是实现特征的位移（shifting）操作。
    帧拼接或上下文拼接。为了让模型能够捕获到语音特征的上下文信息对每一帧特征进行扩展，
    即将当前帧的前几帧和后几帧的特征也拼接到当前帧的特征上。
    shift 函数在这里的作用就是实现这个拼接过程中不同偏移量特征的对齐。
    """
    #x: 输入的 PyTorch Tensor，形状通常是 (序列长度, 特征维度)。
    #n: 位移的步长。
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)


def concat_feat(x, concat_n):
    """功能： 对语音特征进行上下文拼接（或称作帧拼接、帧扩展）。
    这是语音识别中常见的特征工程步骤，
    目的是让模型在处理当前帧时能考虑到前后几帧的信息。
    核心思想： 对于序列中的每一个时间步 t，我们希望它的新特征是 [feat(t - offset_2), ..., feat(t - 1), feat(t), feat(t + 1), ..., feat(t + offset_2)]。
    """
    #x: 输入的特征 Tensor，形状通常是 (序列长度, 原始特征维度)
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    #(序列长度, 原始特征维度)
    x = x.repeat(1, concat_n)
    #：将原始特征 x 在特征维度上重复 concat_n 次。
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    """
    调用 x.permute(1, 0, 2) 意味着：
    原始的 dim0 会变成新 Tensor 的 dim1。
    原始的 dim1 会变成新 Tensor 的 dim0。
    原始的 dim2 保持不变，依然是新 Tensor 的 dim2。
    """
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
    """
    split: 字符串，可以是 'train'、'val' (验证集) 或 'test'。
    train_ratio定义了从 train_split.txt 中读取的语音片段中，有多少比例用于训练集，剩余的用于验证集。
    例如，0.8 意味着 80% 用于训练，20% 用于验证。
    """
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    """一个大型的空 Tensor，用于存储所有语音片段的拼接特征。
    (仅限训练模式) 一个大型的空 Tensor，用于存储所有电话音素标签。
    """
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        #使用 tqdm 库来显示一个美观的进度条，让你知道数据处理的进展。
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]

    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X

定义了一个名为 LibriDataset 的 PyTorch 自定义数据集类

In [46]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

模型（这里可以改进，多多思考，探寻一下）

一些基础知识，一定要掌握
归一化
归一化是一种数据预处理技术，其核心思想是将数据按比例缩放到一个特定的范围，或者将数据调整为具有特定的分布。
它的目的是消除数据特征之间量纲、数值范围的差异，使得不同特征对模型的影响力处于一个更公平的地位。
归一化的实现方式：
1.Min-Max 归一化，就是几何分布占比的大小
2.Z-score 归一化，有点类似于三大分布的那个预测
3.Batch Normalization (批量归一化) 这个模型用到的
and so on
主要用于：
1.消除量纲影响
2.加速模型收敛
3.防止数值问题（例如梯度爆炸/消失）
说白了归一化是让你的模型学习得更快、更稳定、更准确的关键预处理步骤。
        

Dropout
一种正则化技术，通过在训练时随机地将一部分神经元的输出设置为零来防止过拟合，迫使网络学习更鲁棒的特征。
        
优化器
所有优化器都基于梯度下降的原理，即参数更新的方向与损失函数的梯度方向相反。
一. 随机梯度下降（SGD）：
是什么： SGD 是最基础也是最核心的优化算法之一。它的“随机”体现在：它不像传统的批量梯度下降（Batch Gradient Descent）那样在计算梯度时使用整个训练数据集，而是每次只随机选择一个样本或一小批样本（mini-batch）来计算梯度并更新参数。
优点： 1.计算效率高： 每次更新只处理少量样本，计算开销小，尤其适用于大数据集。2.跳出局部最优： 由于每次更新的梯度带有一定的随机性（噪音），这有助于 SGD 跳出浅层的局部最优解，找到更好的全局最优解。
二. 自适应学习率优化器（Adam）
结合了 RMSprop和（Root Mean Square Propagation）Momentum（动量）的思想。

选择建议：Adam 通常是首选，特别是当你开始一个新项目时。它表现稳定，收敛快。SGD 配合动量和学习率调度器，在某些情况下（如图像分类的 ResNet）可以达到比 Adam 更好的泛化性能，但需要更精细的调参。
        
Batch Size（批量大小）：
Batch Size 是指在神经网络训练过程中，每次迭代（一次参数更新）所使用的样本数量。
怎么确定 Batch Size？ 硬件限制（GPU 显存）： 这是最主要的限制因素。
对于大多数深度学习任务，通常从 32 或 64 开始是一个不错的选择。
对于图像识别等任务，常常会使用更大的 Batch Size，如 128, 256。
学习率（Learning Rate）是什么？
它决定了在每次迭代中，模型参数（权重和偏置）沿着损失函数梯度方向更新的步长大小。
确定学习率是一个经验性很强的过程。对于 Adam 优化器，0.001 通常是一个很好的起始点。对于 SGD，可能需要尝试 0.01、0.1 甚至 0.5。如果你在一个预训练模型的基础上进行微调（Fine-tuning），通常会使用一个非常小的学习率（比如 10 −5或 10 −6）。这是因为预训练模型已经学习到了很多有用的特征，我们只想对其进行微小的调整，而不是颠覆性的改变。
调整模型超参数：
hidden_layers和hidden_dim： 尝试不同的隐藏层数量和每层维度。增加层数和维度可以提高模型容量，但可能导致过拟合或训练困难。
concat_nframes：调整上下文拼接的帧数。不同的任务可能需要不同长度的上下文。
学习率（Learning Rate）：这是最重要的超参数之一。太高可能导致训练发散，太低可能导致收敛过慢。通常会使用学习率调度器（Learning Rate Scheduler）来动态调整学习率。
优化器（Optimizer）： 除了默认的 SGD，可以尝试 Adam、RMSprop 等更先进的优化器，它们通常收敛更快，表现更好。
批量大小（Batch Size）： 不同的批量大小会影响训练的稳定性和收敛速度。
损失函数和评估指标：对于分类任务，nn.CrossEntropyLoss 是标准的损失函数。
评估指标通常是准确率（Accuracy），但对于不平衡数据集，F1-Score、精确率（Precision）和召回率（Recall可能更具参考价值。
训练策略：早停： 监控验证集性能，当验证集性能在连续几个 epoch 没有提升时停止训练，防止过拟合。
学习率调度器（Learning Rate Scheduler）：例如 torch.optim.lr_scheduler.StepLR (每N个epoch降低学习率)、ReduceLROnPlateau (当指标停止改善时降低学习率) 等。
以下是一些可以替代 MLP 来更好地完成任务的模型：循环神经网络 (Recurrent Neural Networks, RNN) 及其变体，RNN 天生适合处理序列数据，因为它们内部有循环结构，能够保留“记忆”或“状态”，从而捕获时间序列中的依赖关系。

In [48]:
import torch.nn as nn

class BasicBlock(nn.Module):
    """
    BasicBlock 是一个简单的神经网络模块，可以被看作是构建更复杂网络的基本“砖块”。
    感知机模型是一种二分类的线性模型，是更复杂神经网络的基础。
    """
    def __init__(self, input_dim, output_dim ,dropout_p=0.5):
        super(BasicBlock, self).__init__()
        # 这里面的TODO已经弄完了，之后可以按上面的Markdown进行改进。
        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),#线性层：将输入维度映射到输出维度
            nn.BatchNorm1d(output_dim), #添加批量归一化
            nn.ReLU(),#激活函数：引入非线性
            nn.Dropout(dropout_p)
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    """
    完整的多层感知机模型

    """
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            # 第一层：从输入维度到隐藏层维度
            BasicBlock(input_dim, hidden_dim),
            # 中间隐藏层：根据 hidden_layers 参数堆叠 BasicBlock
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            # 最后一层：从隐藏层维度到最终输出维度（类别数）
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        #定义了数据如何通过整个分类器，输入 x 依次通过 fc 序列中定义的所有层
        x = self.fc(x)
        return x

完整的多层感知机模型
Classifier 类是整个分类模型的主体，它使用 BasicBlock 作为构建其隐藏层的组件。
核心思想： 如果预测错误，就根据错误的程度和输入特征的方向来调整权重，使得下次再遇到类似输入时，能够做出正确的预测。如果预测值过低，就增加相关的权重；如果预测值过高，就减小相关的权重。
解决线性问题，无法解决非线性问题。著名的"异或问题"就是感知机无法解决的典型非线性问题。对噪声敏感，输出不是概率：感知机的输出是离散的（0 或 1），无法给出输入属于某个类别的概率。
多层感知机（Multi-Layer Perceptron, MLP）： 通过增加隐藏层（hidden layers）和使用非线性激活函数（如 Sigmoid, ReLU 等），多层感知机克服了单层感知机无法处理非线性可分问题的缺点。这是目前广泛使用的前馈神经网络的基础。
上面提供的 Classifier 模型就是一个典型的多层感知机。作为独立的、单层的模型，感知机现在已经不是主流了。单层感知机因其局限性已不再是主流的机器学习模型。但是，它所蕴含的核心思想是所有现代深度学习网络的基础，尤其是多层感知机（MLP）和更复杂的神经网络（如卷积神经网络 CNN, 循环神经网络 RNN）都是在其概念上发展而来的。